In [20]:
#CV data formatting code
import statsmodels as sm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.api import VAR
from sklearn.preprocessing import MinMaxScaler

In [132]:
x = pd.read_csv('/Users/andrewslottje/Documents/cv_x.csv', parse_dates=True)
x = x.rename(columns={"[TimeStamp]":"Timestamp"})
y = pd.read_csv('/Users/andrewslottje/Documents/cv_y.csv', parse_dates=True)

In [133]:
#create new variables for stationarity
x.index = pd.DatetimeIndex(x.Timestamp)
x['ETHspread_d']=np.log(x['ETHspread']/x['ETHspread'].shift(1))
x['ETHvolume_d']=np.log(x['ETHvolume']/x['ETHvolume'].shift(1))
x.loc[x.ETHvolume_d.isnull(),'ETHvolume_d']=0.0
x.loc[x.ETHspread_d.isnull(),'ETHspread_d']=0.0
x['ETHreturnindicator']=np.sign(x['ETHreturn'])
x=x[~x.isnull()]
data = x[['ETHreturn','ETHspread_d','ETHvolume_d','XRPreturn','LTCreturn','DASHreturn','XMRreturn','ETHreturnindicator']]

/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:3: RuntimeWarning: invalid value encountered in log
  app.launch_new_instance()
/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:4: RuntimeWarning: invalid value encountered in log


In [134]:
#fit VAR using train data VAR (we use CV data here for the covariates and will adjust this)
model=VAR(data)
results = model.fit(3)
data['ETHpredict']=results.fittedvalues['ETHreturn']
data['ETHreturn_resid']=(data['ETHreturn']-results.fittedvalues['ETHreturn'])
#include next-period prediction from current filtration
data['ETHpredict']=data['ETHpredict'].shift(1)

/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-cop

In [135]:
#generate lagged data
for i in ['ETHreturn','ETHspread_d','ETHvolume_d','XRPreturn','LTCreturn','DASHreturn','XMRreturn','ETHreturn_resid']:
#for i in ['ETHreturn','ETHspread_d','ETHvolume_d','XRPreturn','LTCreturn','DASHreturn','XMRreturn']:
    #generate 10 lags of data
    for j in range(1,10):
        data[str(i)+"_l"+str(j)]=data[str(i)].shift(j)

data=data.drop(['ETHreturnindicator'],axis=1)
data['ETHreturnindicator']=0.0
data.loc[data.ETHreturn>.005,'ETHreturnindicator']=1.0
data['ETHreturnindicator']=data['ETHreturnindicator'].shift(-1)
#drop nan rows
data=data.drop(data.index[0:14])
data=data.drop(data.index[-1:])

/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [100]:
#see comment in train set
#for i in data.columns.values[:-1]:
#    data[i]=MinMaxScaler(feature_range=(-1, 1)).fit_transform(data[i])

In [139]:
#data.to_csv('/Users/andrewslottje/Documents/train_x.csv')
#data.to_csv('/Users/andrewslottje/Documents/train_x_noresid.csv')
data.to_csv('/Users/andrewslottje/Documents/cv_x_10lags_005.csv')

In [138]:
#class frequency
len(data[data['ETHreturnindicator']==1])/len(data)

0.26191498497209104